In [2]:
from ipsl_dataset import IPSL_DCPP
import torch
import lightning as pl


In [3]:
from pangu import PanguWeather
model = PanguWeather()


/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
from hydra import compose, initialize
from omegaconf import OmegaConf
import hydra

with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")


In [5]:
patch_size = eval(cfg.experiment.patch_size)

In [6]:
train_dataloader = torch.utils.data.DataLoader(IPSL_DCPP('train',cfg.experiment.lead_time_months),batch_size=1,shuffle=False,num_workers=1)
batch = next(iter(train_dataloader))

In [7]:
model = hydra.utils.instantiate(cfg.experiment.module,backbone=hydra.utils.instantiate(cfg.experiment.backbone,patch_size=patch_size),dataset=train_dataloader.dataset)


In [8]:
trainer = pl.Trainer(fast_dev_run=1,max_depth=-1)
output = trainer.fit(model, train_dataloader)


TypeError: Trainer.__init__() got an unexpected keyword argument 'max_depth'

In [ ]:
for name, param in model.named_parameters():
    if param.grad is None:
        print(name)

In [ ]:
out = model.forward(batch)